In [1]:
!pip install anytree

In [2]:
import pandas as pd
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter
from copy import deepcopy

In [3]:
# Leer el archivo CSV
df = pd.read_csv('salida.csv')

In [4]:
# Crear un diccionario para almacenar los nodos
nodes = {}

# Construir los nodos y el árbol
for _, row in df.iterrows():
    parent_name = row['SubClass']
    child_name = row['Class']
    
    # Crear el nodo padre si no existe
    if parent_name not in nodes:
        nodes[parent_name] = Node(parent_name)
    
    # Crear el nodo hijo y asignarlo al nodo padre
    if child_name not in nodes:
        nodes[child_name] = Node(child_name, parent=nodes[parent_name])
    else:
        nodes[child_name].parent = nodes[parent_name]

root_nodes = [node for node in nodes.values() if node.is_root]

In [5]:
# Mostrar el árbol
for root_node in root_nodes:
    for pre, fill, node in RenderTree(root_node, style=AsciiStyle()):
        print(f"{pre}{node.name}")

root
|-- persona
|   |-- tenista
|   +-- atleta
+-- comida
    |-- carne
    |   |-- pollo
    |   |-- vaca
    |   +-- oveja
    |-- vegetal
    |   |-- zanahoria
    |   +-- tomate
    +-- fruta


In [6]:
#Copiar árbol
root_nodes_copy = deepcopy(root_nodes)

root
|-- persona
|   |-- tenista
|   +-- atleta
+-- comida
    |-- carne
    |   |-- pollo
    |   |-- vaca
    |   +-- oveja
    |-- vegetal
    |   |-- zanahoria
    |   +-- tomate
    +-- fruta


In [7]:
# Definir el número máximo de niveles
X = 3

def prune_tree(node, max_depth, current_depth=1):
    # Si estamos al nivel máximo, eliminar todos los hijos
    if current_depth >= max_depth:
        node.children = []
    else:
        # Recursivamente podar cada hijo
        for child in node.children:
            prune_tree(child, max_depth, current_depth + 1)

# Podar el árbol a X niveles
for root in root_nodes_copy:
    prune_tree(root, X)

# Mostrar el árbol podado
for pre, fill, node in RenderTree(root_nodes_copy[0]):
    print("%s%s" % (pre, node.name))

root
├── persona
│   ├── tenista
│   └── atleta
└── comida
    ├── carne
    ├── vegetal
    └── fruta


In [8]:
df_new = pd.read_csv('salida2.csv')

In [10]:
def get_new_class(node, podado_root):
    current_node = node
    while current_node and not any(p.name == current_node.name for p in podado_root.descendants):
        current_node = current_node.parent
    return current_node.name if current_node else None

nodes = {node.name: node for node in root_nodes[0].descendants}
nodes[root.name] = root

print(nodes)

df_new['NewClass'] = df_new['Class'].apply(lambda x: get_new_class(nodes[x], root_nodes_copy[0]))

print(df_new)

{'persona': Node('/root/persona'), 'tenista': Node('/root/persona/tenista'), 'atleta': Node('/root/persona/atleta'), 'comida': Node('/root/comida'), 'carne': Node('/root/comida/carne'), 'pollo': Node('/root/comida/carne/pollo'), 'vaca': Node('/root/comida/carne/vaca'), 'oveja': Node('/root/comida/carne/oveja'), 'vegetal': Node('/root/comida/vegetal'), 'zanahoria': Node('/root/comida/vegetal/zanahoria'), 'tomate': Node('/root/comida/vegetal/tomate'), 'fruta': Node('/root/comida/fruta'), 'root': Node('/root')}
      Text      Class NewClass
0    carne      pollo    carne
1    carne       vaca    carne
2    carne      oveja    carne
3  vegetal  zanahoria  vegetal
4  vegetal     tomate  vegetal
